<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/Esik%C3%A4sittely_3_tiedostoa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Skripti Helsingin sää- ja ilmanlaatutietojen yhdistämiseen ja esikäsittelyyn (KORJATTU 2 - Duplikaatit).

Lataa kolme CSV-tiedostoa FMI:n datasta GitHubista:
1. Säädata (osa 1), Helsinki Kaisaniemi
2. Säädata (osa 2), Helsinki Kaisaniemi
3. Ilmanlaatutata (Otsoni), Helsinki Kallio 2

Yhdistää ja esikäsittelee datan valmiiksi aikasarja-analyysia varten.
Käsittelee mahdolliset duplikaattiaikaleimat yhdistämisen jälkeen.
"""

import pandas as pd
import numpy as np
import requests
import io

print("Vaihe 1: Alustus ja tiedostojen URL-osoitteet")

# --- Tiedostojen URL-osoitteet GitHubissa (raw-versiot) ---
url_weather_1 = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%2011.4.2023%20-%2011.4.2025_4c0a0316-74e0-4792-9854-fd6315fbc965.csv"
url_weather_2 = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%2011.4.2023%20-%2011.4.2025_84370efb-e7a0-48ed-b991-0432c84c1475.csv"
url_aq = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kallio%202_%2011.4.2023%20-%2011.4.2025_8bbe3500-d31d-459d-ac37-16b4bd64a2cc.csv"

# --- Latausfunktio ---
def load_csv_from_url(url):
    """Lataa CSV-tiedoston URL-osoitteesta ja palauttaa Pandas DataFramen."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        csv_data = io.StringIO(response.text)
        try:
            df = pd.read_csv(csv_data, sep=',')
            print(f"Luettu onnistuneesti oletusasetuksin (sep=','): {url}")
            return df
        except Exception as e1:
            print(f"Oletusluku (sep=',') epäonnistui ({e1}), yritetään muita...")
            csv_data.seek(0)
            print(f"Virhe luettaessa CSV-tiedostoa: {url}\nVirhe: {e1}")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Virhe ladattaessa dataa URL-osoitteesta: {url}\nVirhe: {e}")
        return None

# --- Lataa datat ---
print("\nLadataan datatiedostot...")
df_w1_raw = load_csv_from_url(url_weather_1)
df_w2_raw = load_csv_from_url(url_weather_2)
df_aq_raw = load_csv_from_url(url_aq)

if df_w1_raw is None or df_w2_raw is None or df_aq_raw is None:
    print("\nKaikkia tiedostoja ei voitu ladata. Keskeytetään.")
else:
    print("\nKaikki tiedostot ladattu. Jatketaan esikäsittelyyn.")

#==============================================================================
print("\nVaihe 2: Datan esikäsittely (jokainen tiedosto erikseen)")
#==============================================================================

# --- Esikäsittelyfunktio ---
def preprocess_fmi_data(df, data_type_name):
    """Esikäsittelee FMI:n datan: aikaleima, valitsee sarakkeet (KORJATTU)."""
    if df is None: return None
    print(f"\nEsikäsitellään: {data_type_name}")
    original_cols = df.columns.tolist()
    print("Alkuperäiset sarakkeet:", original_cols)

    date_cols = ['Vuosi', 'Kuukausi', 'Päivä']
    time_col = 'Aika [Paikallinen aika]'

    if all(col in original_cols for col in date_cols) and time_col in original_cols:
        try:
            datetime_str_series = df['Vuosi'].astype(str) + '-' + \
                                  df['Kuukausi'].astype(str).str.zfill(2) + '-' + \
                                  df['Päivä'].astype(str).str.zfill(2) + ' ' + \
                                  df[time_col].astype(str)
            df['Timestamp'] = pd.to_datetime(datetime_str_series)
            print("Aikaleima muodostettu sarakkeista: Vuosi, Kuukausi, Päivä, Aika [Paikallinen aika].")
            cols_to_drop = date_cols + [time_col]
            if 'Havaintoasema' in original_cols:
                 cols_to_drop.append('Havaintoasema')
            df = df.drop(columns=cols_to_drop, errors='ignore')
        except Exception as e:
            print(f"Virhe aikaleiman muodostamisessa: {e}. Tarkista sarakkeiden 'Aika [Paikallinen aika]' muoto.")
            return None
    else:
        missing_cols = [col for col in date_cols + [time_col] if col not in original_cols]
        print(f"Tarvittavia aikaleimasarakkeita ei löytynyt. Puuttuvat: {missing_cols}")
        return None

    try:
        df = df.set_index('Timestamp')
    except KeyError:
        print("Virhe: 'Timestamp'-saraketta ei löytynyt indeksiksi asettamista varten.")
        return None

    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')
    print("Sarakkeet muunnettu numeerisiksi (virheet korvattu NaN:lla).")

    df = df.dropna(axis=1, how='all')
    print(f"Sarakkeet jotka jäivät jäljelle: {df.columns.tolist()}")
    return df

# --- Esikäsittele Säädata 1 ---
df_w1 = preprocess_fmi_data(df_w1_raw, "Säädata 1 (Kaisaniemi)")
if df_w1 is not None:
    rename_map_w1 = {
        'Ilman lämpötila keskiarvo [°C]': 'Temperature_W1',
        'Tuulen suunta keskiarvo [°]': 'WindDirection',
        'Keskituulen nopeus keskiarvo [m/s]': 'WindSpeed_W1',
        'Näkyvyys keskiarvo [m]': 'Visibility',
        'Pilvisyys [1/8]': 'Cloudiness', # Tämä jäi edellisestä tulosteesta pois, otetaan mukaan
        'Ilmanpaine merenpinnan tasolla keskiarvo [hPa]': 'Pressure_SeaLevel'
    }
    valid_rename_map_w1 = {k: v for k, v in rename_map_w1.items() if k in df_w1.columns}
    df_w1 = df_w1.rename(columns=valid_rename_map_w1)
    print("Säädata 1 sarakkeet uudelleennimeämisen jälkeen:", df_w1.columns.tolist())
    print(df_w1.head(2))

# --- Esikäsittele Säädata 2 ---
df_w2 = preprocess_fmi_data(df_w2_raw, "Säädata 2 (Kaisaniemi)")
if df_w2 is not None:
    rename_map_w2 = {
        'Lämpötilan keskiarvo [°C]': 'Temperature_W2',
        'Ylin lämpötila [°C]': 'Temperature_Max',
        'Alin lämpötila [°C]': 'Temperature_Min',
        'Keskituulen nopeus [m/s]': 'WindSpeed_W2',
        'Tuulen suunnan keskiarvo [°]': 'WindDirection_W2',
        'Ilmanpaineen keskiarvo [hPa]': 'Pressure_W2'
    }
    valid_rename_map_w2 = {k: v for k, v in rename_map_w2.items() if k in df_w2.columns}
    df_w2 = df_w2.rename(columns=valid_rename_map_w2)
    print("Säädata 2 sarakkeet uudelleennimeämisen jälkeen:", df_w2.columns.tolist())
    print(df_w2.head(2))

# --- Esikäsittele Ilmanlaatu (Kallio 2) ---
df_aq = preprocess_fmi_data(df_aq_raw, "Ilmanlaatu (Kallio 2)")
if df_aq is not None:
    aq_cols_to_keep_and_rename = {
        'Otsoni [µg/m3]': 'Ozone',
        'Hengitettävät hiukkaset <10 µm [µg/m3]': 'PM10',
        'Pienhiukkaset <2.5 µm [µg/m3]': 'PM25',
        'Typpidioksidi [µg/m3]': 'NO2',
        'Typpimonoksidi [µg/m3]': 'NO',
        'Hiilimonoksidi [µg/m3]': 'CO', # Lisätään tämäkin, jos löytyy
        'Rikkidioksidi [µg/m3]': 'SO2',
        'Musta hiili [µg/m3]': 'BlackCarbon' # Lisätään tämäkin
    }
    cols_to_select = [k for k in aq_cols_to_keep_and_rename.keys() if k in df_aq.columns]
    if 'Otsoni [µg/m3]' not in cols_to_select:
         print("VAROITUS: Otsonisaraketta 'Otsoni [µg/m3]' ei löytynyt ilmanlaatudatasta!")
         df_aq = None
    else:
        df_aq_filtered = df_aq[cols_to_select]
        valid_rename_map_aq = {k: v for k, v in aq_cols_to_keep_and_rename.items() if k in cols_to_select}
        df_aq = df_aq_filtered.rename(columns=valid_rename_map_aq)
        print("Ilmanlaatudata sarakkeet valinnan ja uudelleennimeämisen jälkeen:", df_aq.columns.tolist())
        print(df_aq.head(2))

#==============================================================================
print("\nVaihe 3: DataFramesien yhdistäminen")
#==============================================================================

if df_w1 is not None and df_w2 is not None and df_aq is not None:
    print("\nYhdistetään säädatat (df_w1 ja df_w2)...")
    df_weather_combined = pd.merge(df_w1, df_w2, left_index=True, right_index=True, how='outer')
    print("Yhdistetyn säädatan muoto:", df_weather_combined.shape)
    print("Yhdistetyn säädatan sarakkeet:", df_weather_combined.columns.tolist())

    print("\nYhdistetään yhdistetty säädata ja ilmanlaatutata (df_aq)...")
    df_final = pd.merge(df_weather_combined, df_aq, left_index=True, right_index=True, how='outer')
    print("Lopullisen yhdistetyn datan muoto:", df_final.shape)
    print("Lopullisen yhdistetyn datan sarakkeet:", df_final.columns.tolist())

    #==========================================================================
    print("\nVaihe 4: Lopullinen siivous ja NaN-arvojen käsittely")
    #==========================================================================

    # --- Varmista aikajärjestys ---
    df_final = df_final.sort_index()

    # --- TARKISTA JA KÄSITTELE DUPLIKAATTI-INDEKSIT (KORJATTU) ---
    if not df_final.index.is_unique: # Tarkempi tapa tarkistaa duplikaatit
        num_duplicates = df_final.index.duplicated().sum()
        print(f"\nVaroitus: Löydettiin {num_duplicates} duplikaatti-indeksiä.")
        # Strategia: Lasketaan keskiarvo duplikaattien yli.
        print("Käsitellään duplikaatit laskemalla keskiarvo...")
        df_final = df_final.groupby(level=0).mean() # Käytä level=0 viittaamaan indeksiin
        print(f"Duplikaatit käsitelty. Uusi muoto: {df_final.shape}")
        if df_final.index.has_duplicates:
             print("VAROITUS: Duplikaatteja jäi käsittelyn jälkeen!")
        else:
             print("Duplikaatti-indeksit poistettu onnistuneesti.")
    else:
        print("\nIndeksi on uniikki, ei duplikaatteja.")

    # --- Varmista tasainen tuntifrekvenssi ---
    print("\nVarmistetaan tasainen tuntifrekvenssi...")
    if not df_final.empty:
      min_ts, max_ts = df_final.index.min(), df_final.index.max()
      print(f"Aika-alue datassa: {min_ts} - {max_ts}")
      # Luo täydellinen tuntialue
      full_time_range = pd.date_range(start=min_ts, end=max_ts, freq='h')
      print(f"Odotettu tuntien määrä: {len(full_time_range)}")
      # Uudelleenindeksoi DataFrame tällä alueella
      df_final = df_final.reindex(full_time_range)
      print(f"DataFrame uudelleenindeksoitu tasaiselle tuntifrekvenssille. Uusi muoto: {df_final.shape}")
    else:
      print("DataFrame on tyhjä, ei voida uudelleenindeksoida.")

    # --- Käsittele puuttuvat arvot (NaN) ---
    print("\nKäsitellään puuttuvat arvot (NaN)...")
    if not df_final.empty:
      nan_counts_before = df_final.isnull().sum()
      print("NaN-arvojen määrä / sarake ENNEN täyttöä:\n", nan_counts_before[nan_counts_before > 0])
      df_final = df_final.ffill()
      df_final = df_final.bfill()
      nan_counts_after = df_final.isnull().sum().sum()
      if nan_counts_after == 0:
          print("\nKaikki NaN-arvot täytetty onnistuneesti (ffill + bfill).")
      else:
          print(f"\nVaroitus: {nan_counts_after} NaN-arvoa jäi täytön jälkeen. Tarkista data.")
          print("Sarakkeet joissa NaN:\n", df_final.isnull().sum()[df_final.isnull().sum() > 0])
    else:
      print("DataFrame on tyhjä, NaN-arvojen käsittelyä ei suoriteta.")

    # --- Lopullinen tarkistus ---
    if not df_final.empty:
      print("\nLopullisen DataFramen 5 ensimmäistä riviä:")
      print(df_final.head())
      print("\nLopullisen DataFramen 5 viimeistä riviä:")
      print(df_final.tail())
      print("\nLopullisen DataFramen perustilastot:")
      print(df_final.describe())
      print("\nLopullisen DataFramen info:")
      df_final.info()
    else:
        print("\nLopullinen DataFrame on tyhjä.")

    #==========================================================================
    print("\nVaihe 5: Tallenna käsitelty data (Valinnainen)")
    #==========================================================================
    if not df_final.empty:
      # output_path = "data/processed/Helsinki_Kaisaniemi_Kallio_Combined_Processed_New.csv"
      # try:
      #     import os
      #     os.makedirs(os.path.dirname(output_path), exist_ok=True)
      #     df_final.to_csv(output_path)
      #     print(f"\nKäsitelty data tallennettu tiedostoon: {output_path}")
      # except Exception as e:
      #     print(f"\nVirhe tallennettaessa käsiteltyä dataa: {e}")
      pass

    print("\nSkripti suoritettu loppuun.")

else:
    print("\nEsikäsittelyä tai yhdistämistä ei voitu suorittaa, koska kaikkia dataja ei ladattu tai esikäsitelty onnistuneesti.")